# 02. Нейросети и PyTorch

## План
1. Готовим обучение
    1. Данные: `Dataset` & `DataLoader` 
    2. Модель: `nn.Module`
    3. Рутина: все остальное
2. Учим
    1. Baseline
    2. Stack more layers
3. I/O


In [ ]:
import torch

## 1. Готовим обучение 

Общий подход к решению задачи на pytorch такой:
1. Подготовить данные, реализовать (или использовать готовый) класс `Dataset`, наследуясь от `torch.utils.data.Dataset`, обернуть его в `torch.utils.data.DataLoader`.
2. Реализовать (или взять ±готовую) модель, наследуясь от `torch.nn.Module`.
3. Приготовить оптимизатор для весов модели (из `torch.optim` или свой) и лосс
4. Написать код для рутины обучения, включающий обработку данных из `DataLoader`, прогон их через модель, вычисление лосса и обновление весов оптимизатором.

### 1.1. Данные: `Dataset` & `DataLoader`

* [Tutorial @ pytorch.org](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)

Класс датасета предоставит нам интерфейс к данным:
* Метод `__getitem__(self, i)` позволяет получить `i`-й элемент обучающей выборки, обычно пару (data, label).
    * Также обязательным является определение метода `__len__(self)`.
* Можно сделать так, чтобы экземпляр класса датасета просто возвращал исходные данные, а можно (нужно) добавить в него аугментирование данных.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

`Dataset` - абстрактный класс, его нельзя использовать напрямую, а только через наследование:

In [ ]:
dataset = Dataset()
dataset[0]

Создадим датасет поверх игрушечных данных с прошлого семинара:

In [ ]:
np.random.seed(1234)
_a = np.random.uniform(1, 5)
_b = np.random.uniform(-3, 3)
_c = np.random.uniform(-3, 3)

num_samples = 1000

xs = np.random.uniform(-3, 3, size=num_samples)
ys_clean = _a * xs ** 2 + _b * xs + _c
ys_noise = np.random.normal(0, 1, size=len(ys_clean))
ys = ys_clean + ys_noise

plt.figure(figsize=(12, 5))
plt.scatter(xs, ys, label="gt", s=5)
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)

In [ ]:
class CustomDataset(Dataset):
    
    def __init__(self, xs, ys):
        super().__init__()
        
        if len(xs) != len(ys):
            raise ValueError(f"lens mismatch: {len(xs)} != {len(ys)}")
            
        self.xs = xs
        self.ys = ys
        
    def __len__(self):
        return len(self.xs)
        
    def __getitem__(self, i):
        return (self.xs[i], self.ys[i])
    
    @staticmethod
    def collate_fn(items_list):
        xs = torch.zeros(len(items_list), 1)
        ys = torch.zeros(len(items_list), 1)

        for i, (x, y) in enumerate(items_list):
            xs[i] = x
            ys[i] = y

        return xs, ys

Метод `collate_fn` нужен не столько для самого датасета, сколько для оборачивания его в `DataLoader` - об этом чуть ниже.

In [ ]:
dataset = CustomDataset(xs, ys)
dataset[0]

In [ ]:
dataset[1]

In [ ]:
len(dataset)

In [ ]:
dataset[100]

По датасету можно итерироваться (но вам это вряд ли будет нужно часто):

In [ ]:
for x in dataset:
    print(x)

Теоретически, для обучения достаточно уже объекта типа `Dataset`. Однако, для удобства и для автоматизации процессов перемешивания данных, формирования батчей и использования многопоточности есть удобный класс `DataLoader`:

In [ ]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True, 
    collate_fn=dataset.collate_fn
)

"Длина" даталоадера - это количество батчей:

In [ ]:
len(dataloader)

К даталоадеру нельзя обращаться по индексу, но можно итерироваться по нему:

In [ ]:
dataloader[0]

In [ ]:
for batch in dataloader:
    xs, ys = batch
    print(xs.shape, ys.shape)

Как именно происходит сборка батчей, покажем, реализовав свой игрушечный даталоадер с аналогичным функционалом:

**Задание**:
Реализовать метод `__getitem__(self, i)`, который должен возвращать i-й батч. 
* Батч должен быть списком с числом элементов = равным числу элементов, возвращаемых датасетом при обращении по индексу (обычно 2 - данные и лейблы, но есть варианты).
    * Каждый из элементов содержит не отдельный объект, а склеенный из отдельных объектов тензов
    * Длина каждого = `batch_size`
* Для сборки батча из отдельных элементов датасета используйте метод `self.dataset.collate_fn`

In [ ]:
class MyDataLoader:
    
    def __init__(self, dataset, batch_size, collate_fn):
        self.dataset = dataset
        self.batch_size = batch_size
        self.collate_fn = collate_fn
        
        self.indices = np.arange(len(dataset))
        
    def __len__(self):
        return len(dataset) // self.batch_size
    
    def __getitem__(self, i):
        # YOUR CODE HERE
        
        # indices = ...
        # items = ...
        # batch = ...
        
        # END OF YOUR CODE
        
        return batch

In [ ]:
my_dataloader = MyDataLoader(dataset, batch_size=32, collate_fn=dataset.collate_fn)

In [ ]:
batch = my_dataloader[0]

assert len(batch) == 2
assert batch[0].shape == (32, 1)
assert batch[1].shape == (32, 1)

Про параметры `DataLoader`-а, которые мы сегодня не трогали (`pin_memory`, `num_workers`, ...), поговорим в другой раз.

### 1.2. Модель: `nn.Module`

Нейросетевые модели состоят из слоев, которые применяются ко входу (обычно) последовательно.
Каждый слой должен быть наследником `torch.nn.Module`, чтобы сам pytorch понимал: перед ним слой нейросети, у него есть параметры, его надо уметь дифференцировать, и т.д.

In [ ]:
import torch.nn

**Задание:**
Реализовать недостающие куски кода в методах `__init__()` и `forward()`.
* В `__init__()` должны быть инициализированы матрица `self.weights` (`out_dim x in_dim`) и вектор `bias` (или `None`).
* В `forward()` они должны быть применены ко входу `x` (`batch x in_dim`).

**NB**: Помните, что обычно обработка данных моделью происходит по батчам, т.е. даже если на вход придет 1 объект, у него будет размерность (`batch x in_dim`).

In [ ]:
class CustomLinear(torch.nn.Module):
    
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        
        # YOUR CODE HERE
        
        self.bias = ...
        self.weights = ...
        
        # END OF YOUR CODE
        
    def forward(self, x):
        
        # YOUR CODE HERE
        
        
        # END OF YOUR CODE
        
        return output
    
    def __repr__(self):
        return f"CustomLinear({self.weights.shape[1]}, {self.weights.shape[0]}, bias={self.bias is not None})"

In [ ]:
linear = CustomLinear(8, 1)

assert isinstance(linear.weights, torch.nn.Parameter)
assert isinstance(linear.bias, torch.nn.Parameter)
assert linear.weights.shape == (1, 8)
assert linear.bias.shape == (1,)

Посмотрим, какие атрибуты и методы есть у нашего класса при наследовании от `nn.Module`.

Во-первых, доступ к обучаемым (и не только) параметрам:

In [ ]:
for p in linear.parameters():
    print(p)
    print()

In [ ]:
for p in linear.named_parameters():
    print(p)
    print()

In [ ]:
linear.state_dict()

Для удобства чтения и отладки, часто полезно определить метод `__repr__()` для информативного вывода самого объекта:

In [ ]:
print(linear)

Важными полями являются индикатор `.training`: он показывает, в каком режиме находится модель - обучения или инференса.

**Вопрос**: зачем?

In [ ]:
linear.training

In [ ]:
linear.eval()
linear.training

In [ ]:
linear.train()
linear.training

**NB**: Выход из режима `training` не отключает вычисление градиентов!

Как мы уже говорили в прошлый раз, вычисления можно производить не только в одиночной точности; для этого необходимо (но не всегда достаточно) привести все веса к соответствующему типу. Наследование от класса `nn.Module` позволяет сделать это одной командой:

In [ ]:
linear.weights.dtype

In [ ]:
linear = linear.half()

In [ ]:
linear.weights.dtype

In [ ]:
linear = linear.float()

А вот что pytorch из коробки делать не позволяет, так это узнать, на каком устройстве лежит наша модель:

In [ ]:
device = torch.device("cpu")
# device = torch.device("cuda:0")

In [ ]:
linear = linear.to(device)

In [ ]:
linear.device

In [ ]:
linear.weights.device

Теперь попробуем собственно применить нашу модель:

In [ ]:
x = torch.randn(32, 8)

In [ ]:
y = linear(x)
y.shape

In [ ]:
x = torch.randn(32, 9)
y = linear(x)
y.shape

### 1.3. Рутина: все остальное

#### 1.3.1. Оптимизатор

In [ ]:
import torch.optim

In [ ]:
optimizer = torch.optim.SGD(linear.parameters(), lr=1e-4)

In [ ]:
print(optimizer)

In [ ]:
optimizer.param_groups

#### 1.3.2. Лосс

Можно написать самому:

In [ ]:
def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

In [ ]:
xs, ys_true = next(iter(dataloader))

In [ ]:
ys_pred = torch.randn_like(ys_true)

In [ ]:
mse_loss(ys_true, ys_pred)

Можно использовать готовые:

In [ ]:
from torch.nn.functional import mse_loss as torch_mse_loss

In [ ]:
torch_mse_loss(ys_true, ys_pred)

#### 1.3.3. Рутина обучения

**Задание:** Дописать функцию для обучения.
* Получение предсказаний моделью для объектов из батча
* Подсчет лосса
* Обновление весов по вызова backprop

In [ ]:
def train_epoch(model, dataloader, optimizer, loss_fn, epoch):
    model.train()
    
    losses = []
    for batch in dataloader:
        xs, ys_true = batch
        
        # YOUR CODE HERE
        
        # ...
        # ...
        
        # END OF YOUR CODE
        
        losses.append(loss.item())
    
    return np.mean(losses)

На валидации будем еще и сохранять результат предсказаний - для визуализации:

In [ ]:
def val_epoch(model, dataloader, loss_fn):
    model.eval()
    
    losses = []
    preds = []
    for batch in dataloader:
        xs, ys_true = batch
        with torch.no_grad():
            ys_pred = model(xs)
        
        loss = loss_fn(ys_pred, ys_true)        
        losses.append(loss.item())
        
        preds.append(ys_pred.numpy())
    
    preds = np.concatenate(preds, axis=0)
    return np.mean(losses), preds

## 2. Учим

### 2.1. Baseline

Начнем с обучения 1 полносвязного слоя, по сути - аппроксимируем данные прямой.

In [ ]:
import tqdm

In [ ]:
num_epochs = 128
lr = 8e-4
batch_size = 8

train_size = 800

In [ ]:
xs = np.random.uniform(-3, 3, size=num_samples)
ys_clean = _a * xs ** 2 + _b * xs + _c
ys_noise = np.random.normal(0, 1, size=len(ys_clean))
ys = ys_clean + ys_noise

train_dataset = CustomDataset(xs[:train_size], ys[:train_size])
val_dataset = CustomDataset(xs[train_size:], ys[train_size:])

In [ ]:
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    collate_fn=train_dataset.collate_fn, 
    drop_last=True
)

val_dataloader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    collate_fn=train_dataset.collate_fn, 
    drop_last=False
)

In [ ]:
model = CustomLinear(1, 1)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
loss_fn = mse_loss

In [ ]:
losses = []
val_losses = []
val_preds = []
for epoch in tqdm.trange(num_epochs):
    loss = train_epoch(model, train_dataloader, optimizer, loss_fn, epoch)
    losses.append(loss)
    
    val_loss, preds = val_epoch(model, val_dataloader, loss_fn)
    val_losses.append(val_loss)
    val_preds.append(preds)

In [ ]:
plt.figure(figsize=(16, 5))

plt.subplot(1, 2, 1)
plt.plot(losses)
plt.grid(True)
plt.xlabel("epoch")
plt.ylabel("loss")

plt.subplot(1, 2, 2)
plt.plot(val_losses)
plt.grid(True)
plt.xlabel("epoch")
plt.ylabel("val loss")

plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.scatter(xs[train_size:], ys[train_size:], label="true")
plt.scatter(xs[train_size:], val_preds[-1], label="fc_1layer")
plt.legend()
plt.grid()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
fc_1layer_train_losses = losses
fc_1layer_val_losses = val_losses
fc_1layer_preds = preds

### 2.2. Stack more layers

In [ ]:
from torch.nn import Sequential
from torch.nn import ReLU

**Задание**: соберите сеть из двух полносвязных слоев размерами (1, 4) и (4, 1); добавьте между слоями нелинейность ReLU.

In [ ]:

class MegaModel(torch.nn.Module):

    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()

        self.net = torch.nn.Sequential(
            CustomLinear(in_dim, hidden_dim),
            ReLU(),
            CustomLinear(hidden_dim, out_dim)
        )

    def forward(self, x):
        return self.net(x)

# YOUR CODE HERE

model = ...
# END OF YOUR CODE


In [ ]:
for p in model.parameters():
    print(p)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
losses = []
val_losses = []
val_preds = []
for epoch in tqdm.trange(num_epochs):
    loss = train_epoch(model, train_dataloader, optimizer, loss_fn, epoch)
    losses.append(loss)
    
    val_loss, preds = val_epoch(model, val_dataloader, loss_fn)
    val_losses.append(val_loss)
    val_preds.append(preds)

In [ ]:
plt.figure(figsize=(16, 5))

plt.subplot(1, 2, 1)
plt.plot(losses, label="fc_2layers_4h")
plt.plot(fc_1layer_train_losses, label="fc_1layer")
plt.grid(True)
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")

plt.subplot(1, 2, 2)
plt.plot(val_losses, label="fc_2layers_4h")
plt.plot(fc_1layer_val_losses, label="fc_1layer")
plt.grid(True)
plt.legend()
plt.xlabel("epoch")
plt.ylabel("val loss")

plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.scatter(xs[train_size:], ys[train_size:], label="true")
plt.scatter(xs[train_size:], val_preds[-1], label="fc_2layer_4h")
plt.legend()
plt.grid()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
fc_2layer_4h_train_losses = losses
fc_2layer_4h_val_losses = val_losses
fc_2layer_4h_preds = preds

Добавим нейронов в скрытый слой:

In [ ]:

model = MegaModel(1, 8, 1)
model

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
losses = []
val_losses = []
for epoch in tqdm.trange(num_epochs):
    loss = train_epoch(model, train_dataloader, optimizer, loss_fn, epoch)
    losses.append(loss)
    
    val_loss, preds = val_epoch(model, val_dataloader, loss_fn)
    val_losses.append(val_loss)

In [ ]:
plt.figure(figsize=(16, 5))

plt.subplot(1, 2, 1)
plt.plot(losses, label="fc_2layer_8h")
plt.plot(fc_2layer_4h_train_losses, label="fc_2layer_4h")
plt.plot(fc_1layer_train_losses, label="fc_1layer")
plt.grid(True)
plt.legend()
plt.xlabel("epoch")
plt.ylabel("loss")

plt.subplot(1, 2, 2)
plt.plot(val_losses, label="fc_2layer_8h")
plt.plot(fc_2layer_4h_val_losses, label="fc_2layer_4h")
plt.plot(fc_1layer_val_losses, label="fc_1layer")
plt.grid(True)
plt.legend()
plt.xlabel("epoch")
plt.ylabel("val loss")

plt.show()

In [ ]:
plt.figure(figsize=(12, 5))
plt.scatter(xs[train_size:], ys[train_size:], label="true")
plt.scatter(xs[train_size:], preds, label="fc_2layer_8h")
plt.legend()
plt.grid()
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Видимо, что наша модель ведет себя как кусочно-линейная функция. Любопытные визуализации на эту тему можно найти, например, [здесь](http://neuralnetworksanddeeplearning.com/chap4.htmlhttp://neuralnetworksanddeeplearning.com/chap4.html).

## 3. I/O

Обученные веса модели хорошо бы уметь сохранять и загружать для дальнейшего использования.

In [ ]:
print(model)

В Pytorch сохранение и загрузка весов выполняется через `state_dict` модели:

In [ ]:
print(model.state_dict())

### 3.1. Save

In [ ]:
output_fn = "./state_dict.pth.tar"

In [ ]:
with open(output_fn, "wb") as fp:
    torch.save(model.state_dict(), fp)

### 3.2. Load

In [ ]:
model = MegaModel(1,8,1)

In [ ]:
print(model.state_dict())

In [ ]:
with open(output_fn, "rb") as fp:
    state_dict = torch.load(fp, map_location="cpu")
state_dict

In [ ]:
model.load_state_dict(state_dict)

In [ ]:
model.state_dict()

Помимо непосредственно весов, бывает полезно сохранить и состояние других объектов: например, оптимизатора (чтобы продолжить обучении с той же точки):

In [ ]:
def save_checkpoint(model, optimizer, output_fn):
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict()
    }
    
    with open(output_fn, "wb") as fp:
        torch.save(checkpoint, output_fn)
        
def load_checkpoint(checkpoint_fn, model, optimizer):
    with open(checkpoint_fn, "rb") as fp:
        checkpoint = torch.load(fp, map_location="cpu")
    
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
optimizer.param_groups[0]["lr"] = 1e-10
optimizer

In [ ]:
checkpoint_fn = "./checkpoint.pth.tar"

In [ ]:
save_checkpoint(model, optimizer, checkpoint_fn)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
optimizer

In [ ]:
load_checkpoint(checkpoint_fn, model, optimizer)

In [ ]:
optimizer